# XLS-R (Wav2Vec2) CTC Fine-tuning on BAAI/CS-Dialogue (Mandarin–English Code-Switching)

**Goal:** Fine-tune `facebook/wav2vec2-xls-r-300m` with a custom CTC vocabulary built from CS-Dialogue.

**Key points**
- Uses `datasets.Audio` + `cast_column(..., Audio(16000))`.
- Builds mixed EN (A–Z) + Chinese vocab; maps space→`|`.
- Correct padding & metrics (WER/CER).

References:
- CS-Dialogue dataset card (16kHz, structure): https://huggingface.co/datasets/BAAI/CS-Dialogue
- XLS-R-300M model card (16kHz input): https://huggingface.co/facebook/wav2vec2-xls-r-300m
- Datasets audio processing: https://huggingface.co/docs/datasets/en/audio_process
- WER / CER metrics: https://huggingface.co/spaces/evaluate-metric/wer , https://huggingface.co/spaces/evaluate-metric/cer


In [1]:
# installations
!pip -q install   "evaluate==0.4.3" "jiwer==3.0.4"
!pip install "torchcodec==0.7.*" --index-url https://download.pytorch.org/whl/cu126
!pip install -U "peft==0.17.1" "accelerate>=1.1.0"
!pip install -U "bitsandbytes==0.48.1"
!pip install -U "pyctcdecode==0.5.0"
#===============================================================================
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:512"
# os.environ["HF_HUB_DISABLE_XET"] = "1"         # 强制不用 XET 后端
# os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"  # 使用更快更稳的 hf_transfer
from importlib.metadata import version, PackageNotFoundError
import transformers, datasets, evaluate, tokenizers, huggingface_hub, torch, torchaudio, jiwer
import peft
from peft import LoraConfig

def safe_ver(pkg_name):
    try:
        return version(pkg_name)
    except PackageNotFoundError:
        return "(missing)"

# 改用 metadata 查询
print("Transformers  :", safe_ver("transformers"))
print("Datasets      :", safe_ver("datasets"))
print("Evaluate      :", safe_ver("evaluate"))
print("JiWER         :", safe_ver("jiwer"))
print("Tokenizers    :", safe_ver("tokenizers"))
print("HF Hub        :", safe_ver("huggingface_hub"))
print("Torch         :", torch.__version__)
print("Torchaudio    :", torchaudio.__version__)
print("PEFT          :", safe_ver("peft"))
print("Accelerate    :", safe_ver("accelerate"))
print("Bitsandbytes  :", safe_ver("bitsandbytes"))
print("PyCTCDecode   :", safe_ver("pyctcdecode"))
print("PYTORCH_CUDA_ALLOC_CONF =", os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))
# 简单构建一个 LoRA 配置，验证 API 可用
test_cfg = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
                      target_modules=["q_proj","k_proj","v_proj","out_proj"])
print(test_cfg)


Looking in indexes: https://download.pytorch.org/whl/cu126
Transformers  : 4.57.1
Datasets      : 4.0.0
Evaluate      : 0.4.3
JiWER         : 3.0.4
Tokenizers    : 0.22.1
HF Hub        : 0.35.3
Torch         : 2.8.0+cu126
Torchaudio    : 2.8.0+cu126
PEFT          : 0.17.1
Accelerate    : 1.11.0
Bitsandbytes  : 0.48.1
PyCTCDecode   : 0.5.0
PYTORCH_CUDA_ALLOC_CONF = expandable_segments:True,max_split_size_mb:512
LoraConfig(task_type=None, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'out_proj', 'q_proj', 'v_proj', 'k_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, u

In [2]:
!python -m bitsandbytes


=================== bitsandbytes v0.48.1 ===================
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
  libc: glibc-2.35
Python: 3.12.12
PyTorch: 2.8.0+cu126
  CUDA: 12.6
  HIP: N/A
  XPU: N/A
Related packages:
  accelerate: 1.11.0
  diffusers: 0.35.2
  numpy: 1.26.4
  pip: 24.1.2
  peft: 0.17.1
  safetensors: 0.6.2
  transformers: 4.57.1
  triton: 3.4.0
  trl: not found
PyTorch settings found: CUDA_VERSION=126, Highest Compute Capability: (8, 0).
Checking that the library is importable and CUDA is callable...
SUCCESS!


In [ ]:
import os
import re
import tarfile
import json
from pathlib import Path
from collections import Counter
from typing import Dict, List
import numpy as np
import torch
from datasets import Dataset, DatasetDict, Audio, Features, Value
from huggingface_hub import hf_hub_download
from transformers import AutoFeatureExtractor, AutoModelForCTC, TrainingArguments, Trainer, Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import evaluate

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

BASE_DIR = Path('/content') if Path('/content').exists() else Path.cwd()
DATA_ROOT = BASE_DIR / 'cs_dialogue'
AUDIO_DIR = DATA_ROOT / 'short_wav'
INDEX_DIR = DATA_ROOT / 'data' / 'index' / 'short_wav'
VOCAB_DIR = DATA_ROOT / 'custom_vocab'
for p in [DATA_ROOT, AUDIO_DIR, INDEX_DIR, VOCAB_DIR]:
    p.mkdir(parents=True, exist_ok=True)


# increase to 19 for full short_wav
NUM_SHARDS = int(os.environ.get('CS_NUM_SHARDS', 19))

## 1) Download index & audio shards (short_wav)

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from pathlib import Path
import os
import shutil
import tarfile
import glob

REPO_ID = "BAAI/CS-Dialogue"        # 数据集仓库
# 你已有：DATA_ROOT = Path('/content/cs_dialogue'); AUDIO_DIR = DATA_ROOT/'short_wav'; INDEX_DIR = DATA_ROOT/'index'/'short_wav'


def download_index():
    files = [
        "data/index/short_wav/train/text",
        "data/index/short_wav/train/wav.scp",
        "data/index/short_wav/dev/text",
        "data/index/short_wav/dev/wav.scp",
        "data/index/short_wav/test/text",
        "data/index/short_wav/test/wav.scp",
    ]
    local = []
    for fp in files:
        # ★ 关键：直接落到 INDEX_DIR/ 相对目录；禁用 symlink
        dst_dir = INDEX_DIR / \
            Path(fp).parent.relative_to("data/index/short_wav")
        dst_dir.mkdir(parents=True, exist_ok=True)
        src = hf_hub_download(
            repo_id=REPO_ID,
            filename=fp,
            repo_type="dataset",
            local_dir=str(DATA_ROOT),   # 关键：交给 local_dir 来还原相对层级
        )
        local.append(Path(src))
    print("Index ready:", local)
    return local


def download_shards(n=19):
    got = []
    for i in range(n):
        rel = f"data/short_wav/short_wav.tar.gz{i:02d}"
        src = hf_hub_download(
            repo_id=REPO_ID,
            filename=rel,
            repo_type="dataset",
            local_dir=str(DATA_ROOT),   # 落到 DATA_ROOT/rel
        )
        dst = Path(src)
        print("Downloaded:", dst)
        assert dst.exists() and dst.stat(
        ).st_size > 0, f"missing or empty part: {dst}"
        got.append(dst)
    # 关键：分片实际在 DATA_ROOT/data/short_wav 下
    print("In data/short_wav:", sorted(p.name for p in (DATA_ROOT /
          'data'/'short_wav').glob('short_wav.tar.gz*')))
    return got


def concat_parts(parts, out_file: Path):
    """把 *.tar.gz00.. 拼接成一个完整 tar.gz"""
    parts = sorted(parts, key=lambda p: p.name)
    if not parts:
        raise RuntimeError("no parts provided to concat_parts")
    # 逐个断言存在
    for p in parts:
        if not Path(p).exists():
            raise FileNotFoundError(f"part not found on disk: {p}")
    out_file.parent.mkdir(parents=True, exist_ok=True)
    total = 0
    with open(out_file, "wb") as w:
        for p in parts:
            with open(p, "rb") as r:
                shutil.copyfileobj(r, w)
            sz = Path(p).stat().st_size
            total += sz
            print(f"  appended {Path(p).name} ({sz/1e6:.1f} MB)")
    print(f"==> concatenated -> {out_file} (~{total/1e9:.2f} GB)")


def extract_concatenated_tar_gz(out_dir: Path, parts=None):
    """
    解压流程：
    1) 如未给 parts，则自动从 DATA_ROOT 扫描 short_wav.tar.gz[0-9][0-9]
    2) 拼成 short_wav.tar.gz
    3) 正常 tar 解压到 out_dir
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    # ★ 自动发现分片（避免用了旧的 tars 变量）
    if parts is None:
        # 同时在 DATA_ROOT 根 & DATA_ROOT/data/short_wav 下搜索
        candidates = sorted((DATA_ROOT).glob("short_wav.tar.gz[0-9][0-9]")) \
            + sorted((DATA_ROOT / "data" /
                     "short_wav").glob("short_wav.tar.gz[0-9][0-9]"))
    else:
        candidates = list(map(Path, parts))

    if not candidates:
        raise RuntimeError(
            f"No split parts found in {DATA_ROOT}. Expected files like short_wav.tar.gz00")

    merged = DATA_ROOT / "short_wav.tar.gz"

    if merged.exists():
        merged.unlink()  # 清除此前拼接失败的半成品
    concat_parts(candidates, merged)

    # 解压
    with tarfile.open(merged, "r:gz") as tf:
        tf.extractall(out_dir, filter="data")  # 3.12+ 推荐
    print("Extracted ->", out_dir)

    # 可选：节省空间
    try:
        merged.unlink()
    except Exception:
        pass


# === 调用顺序 ===
_ = download_index()
_ = download_shards(NUM_SHARDS)                 # e.g. NUM_SHARDS=2

# ★ 完整性检查：必须 19 片全部到位
expected = [f"short_wav.tar.gz{i:02d}" for i in range(19)]
have = sorted(p.name for p in (DATA_ROOT/'data' /
              'short_wav').glob('short_wav.tar.gz*'))
missing = [x for x in expected if x not in have]
assert not missing, f"缺少分片：{missing}。请把 NUM_SHARDS 设为 19 或补齐后再解压。"

extract_concatenated_tar_gz(AUDIO_DIR, None)   # 让它自己扫描分片再拼接

text: 0.00B [00:00, ?B/s]

wav.scp: 0.00B [00:00, ?B/s]

text: 0.00B [00:00, ?B/s]

wav.scp: 0.00B [00:00, ?B/s]

text: 0.00B [00:00, ?B/s]

...

Downloaded: /content/cs_dialogue/data/short_wav/short_wav.tar.gz16


data/short_wav/short_wav.tar.gz17:   0%|          | 0.00/524M [00:00<?, ?B/s]

Downloaded: /content/cs_dialogue/data/short_wav/short_wav.tar.gz17


data/short_wav/short_wav.tar.gz18:   0%|          | 0.00/158M [00:00<?, ?B/s]

Downloaded: /content/cs_dialogue/data/short_wav/short_wav.tar.gz18
In data/short_wav: ['short_wav.tar.gz00', 'short_wav.tar.gz01', 'short_wav.tar.gz02', 'short_wav.tar.gz03', 'short_wav.tar.gz04', 'short_wav.tar.gz05', 'short_wav.tar.gz06', 'short_wav.tar.gz07', 'short_wav.tar.gz08', 'short_wav.tar.gz09', 'short_wav.tar.gz10', 'short_wav.tar.gz11', 'short_wav.tar.gz12', 'short_wav.tar.gz13', 'short_wav.tar.gz14', 'short_wav.tar.gz15', 'short_wav.tar.gz16', 'short_wav.tar.gz17', 'short_wav.tar.gz18']
  appended short_wav.tar.gz00 (524.3 MB)
  appended short_wav.tar.gz01 (524.3 MB)
  appended short_wav.tar.gz02 (524.3 MB)
  appended short_wav.tar.gz03 (524.3 MB)
  appended short_wav.tar.gz04 (524.3 MB)
  appended short_wav.tar.gz05 (524.3 MB)
  appended short_wav.tar.gz06 (524.3 MB)
  appended short_wav.tar.gz07 (524.3 MB)
  appended short_wav.tar.gz08 (524.3 MB)
  appended short_wav.tar.gz09 (524.3 MB)
  appended short_wav.tar.gz10 (524.3 MB)
  appended short_wav.tar.gz11 (524.3 MB)
  a

In [ ]:
print("INDEX_DIR =", INDEX_DIR)
print("Has train/text?   ->", (INDEX_DIR/'train'/'text').exists())
print("Has train/wav.scp?->", (INDEX_DIR/'train'/'wav.scp').exists())

print("Split parts in DATA_ROOT/data/short_wav:",
      sorted(p.name for p in (DATA_ROOT/'data'/'short_wav').glob('short_wav.tar.gz*'))[:19])

INDEX_DIR = /content/cs_dialogue/data/index/short_wav
Has train/text?   -> True
Has train/wav.scp?-> True
Split parts in DATA_ROOT/data/short_wav: ['short_wav.tar.gz00', 'short_wav.tar.gz01', 'short_wav.tar.gz02', 'short_wav.tar.gz03', 'short_wav.tar.gz04', 'short_wav.tar.gz05', 'short_wav.tar.gz06', 'short_wav.tar.gz07', 'short_wav.tar.gz08', 'short_wav.tar.gz09', 'short_wav.tar.gz10', 'short_wav.tar.gz11', 'short_wav.tar.gz12', 'short_wav.tar.gz13', 'short_wav.tar.gz14', 'short_wav.tar.gz15', 'short_wav.tar.gz16', 'short_wav.tar.gz17', 'short_wav.tar.gz18']


In [ ]:
from pathlib import Path


def print_tree(root: Path, max_depth: int = 3):
    root = Path(root)

    def walk(p: Path, depth: int, prefix: str = ""):
        if depth < 0:
            return
        items = sorted(p.iterdir(), key=lambda x: (
            x.is_file(), x.name.lower()))
        for i, it in enumerate(items):
            is_last = (i == len(items) - 1)
            connector = "└── " if is_last else "├── "
            name = it.name + ("/" if it.is_dir() else "")
            print(prefix + connector + name)
            if it.is_dir():
                next_prefix = prefix + ("    " if is_last else "│   ")
                walk(it, depth - 1, next_prefix)

    root = root.resolve()
    print(root.as_posix())
    walk(root, max_depth)


# 使用：打印 DATA_ROOT 下三层
print_tree(DATA_ROOT, max_depth=4)

/content/cs_dialogue
├── .cache/
│   └── huggingface/
│       ├── download/
│       │   └── data/
│       │       ├── index/
│       │       └── short_wav/
│       └── .gitignore
├── custom_vocab/
├── data/
│   ├── index/
│   │   └── short_wav/
│   │       ├── dev/
│   │       │   ├── text
│   │       │   └── wav.scp
│   │       ├── test/
│   │       │   ├── text
│   │       │   └── wav.scp
│   │       └── train/
│   │           ├── text
│   │           └── wav.scp
│   └── short_wav/
│       ├── short_wav.tar.gz00
│       ├── short_wav.tar.gz01
│       ├── short_wav.tar.gz02
│       ├── short_wav.tar.gz03
│       ├── short_wav.tar.gz04
│       ├── short_wav.tar.gz05
│       ├── short_wav.tar.gz06
│       ├── short_wav.tar.gz07
│       ├── short_wav.tar.gz08
│       ├── short_wav.tar.gz09
│       ├── short_wav.tar.gz10
│       ├── short_wav.tar.gz11
│       ├── short_wav.tar.gz12
│       ├── short_wav.tar.gz13
│       ├── short_wav.tar.gz14
│       ├── short_wav.tar.gz15
│       ├── sho

## 2) Build DatasetDict from wav.scp & text

In [ ]:
# def read_kv(fp: Path):
#     d={}
#     with open(fp, 'r', encoding='utf-8') as f:
#         for line in f:
#             line=line.strip()
#             if not line: continue
#             k,v=line.split(' ',1)
#             d[k]=v
#     return d

# def make_split(split: str):
#     wavscp = read_kv(INDEX_DIR/split/'wav.scp')
#     text   = read_kv(INDEX_DIR/split/'text')
#     ids, paths, trans = [], [], []
#     for uid, wavpath in wavscp.items():

#         shard = Path(wavpath).parts[-2]
#         fname = Path(wavpath).name
#         local = AUDIO_DIR / shard / fname
#         if local.exists() and uid in text:
#             ids.append(uid); paths.append(str(local)); trans.append(text[uid])
#     feats = Features({'id': Value('string'), 'audio': Audio(sampling_rate=16000), 'transcription': Value('string')})
#     return Dataset.from_dict({'id': ids, 'audio': paths, 'transcription': trans}, features=feats)
from pathlib import Path


def resolve_local_audio_path(wavpath: str) -> Path | None:
    """
    把 wav.scp 的路径映射到本地真实存在的文件。
    兼容如下情况：
    - 路径前缀带不带 'data/'
    - 是否出现双重 'short_wav/short_wav'
    - 绝对路径/相对路径
    """
    p = Path(wavpath.strip())

    # 1) 若本身就是绝对路径且存在，直接返回
    if p.is_absolute() and p.exists():
        return p

    candidates: list[Path] = []

    # 2) wav.scp 一般包含 ".../short_wav/..." 这段；抽取从第一处 'short_wav' 之后的尾部
    parts = p.parts
    if "short_wav" in parts:
        i = parts.index("short_wav")
        tail = Path(*parts[i+1:])  # 去掉第一个 'short_wav' 及之前的前缀
        # 情况 A：你的磁盘上是 short_wav/short_wav/WAVE/...
        if (AUDIO_DIR / "short_wav").exists():
            # /.../short_wav/short_wav/...
            candidates.append(AUDIO_DIR / "short_wav" / tail)
        # 情况 B：只有一层 short_wav/WAVE/...
        # /.../short_wav/...
        candidates.append(AUDIO_DIR / tail)
    else:
        # 没出现 short_wav 关键词时，尝试几种常见组合
        # /content/cs_dialogue/<wav.scp里的相对路径>
        candidates.append(DATA_ROOT / p)
        # /content/cs_dialogue/data/<...>
        candidates.append(DATA_ROOT / "data" / p)

    # 3) 再补充几种保守候选
    candidates.append(DATA_ROOT / p)
    if str(p).startswith("data/"):
        # /content/cs_dialogue/data/short_wav/...
        candidates.append(DATA_ROOT / str(p))
        # /content/cs_dialogue/short_wav/...
        candidates.append(DATA_ROOT / str(p).replace("data/", "", 1))

    for c in candidates:
        if c.exists():
            return c.resolve()
    return None


def read_kv(fp: Path):
    d = {}
    with open(fp, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Kaldi风格：<key><space><value...>
            k, v = line.split(' ', 1)
            d[k] = v
    return d


def make_split(split: str):
    wavscp = read_kv(INDEX_DIR/split/'wav.scp')
    text = read_kv(INDEX_DIR/split/'text')

    ids, paths, trans = [], [], []
    miss_audio, miss_text = 0, 0

    for uid, wavpath in wavscp.items():
        local = resolve_local_audio_path(wavpath)
        if local is None or not local.exists():
            miss_audio += 1
            continue
        if uid not in text:
            miss_text += 1
            continue

        ids.append(uid)
        paths.append(str(local))
        trans.append(text[uid])

    print(f"[{split}] matched {len(ids)} items "
          f"(missing audio: {miss_audio}, missing text: {miss_text})")

    feats = Features({
        'id': Value('string'),
        'audio': Audio(sampling_rate=16000),
        'transcription': Value('string')
    })
    return Dataset.from_dict(
        {'id': ids, 'audio': paths, 'transcription': trans},
        features=feats
    )


train_ds = make_split('train')
val_ds = make_split('dev')
test_ds = make_split('test')
minds = DatasetDict(train=train_ds, validation=val_ds, test=test_ds)
minds = minds.cast_column('audio', Audio(sampling_rate=16000))
minds

[train] matched 26239 items (missing audio: 0, missing text: 0)
[dev] matched 6186 items (missing audio: 0, missing text: 0)
[test] matched 6257 items (missing audio: 0, missing text: 0)


DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'transcription'],
        num_rows: 26239
    })
    validation: Dataset({
        features: ['id', 'audio', 'transcription'],
        num_rows: 6186
    })
    test: Dataset({
        features: ['id', 'audio', 'transcription'],
        num_rows: 6257
    })
})

In [ ]:
MIN_SEC, MAX_SEC = 0.3, 12.0


def _keep_ok(ex):
    sec = ex["audio"]["array"].shape[0] / ex["audio"]["sampling_rate"]
    return (sec >= MIN_SEC) and (sec <= MAX_SEC)


minds = minds.filter(_keep_ok, num_proc=4)

Filter (num_proc=4):   0%|          | 0/26239 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/6186 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/6257 [00:00<?, ? examples/s]

## 3) Normalize transcripts (EN upper + Chinese)

In [ ]:
CN_RANGE = r"\u4E00-\u9FFF"


def normalize_text(ex):
    t = ex['transcription'].strip().upper()
    t = re.sub(fr"[^{CN_RANGE}A-Z' ]+", "", t)
    t = re.sub(r"\s+", " ", t)
    return {'transcription': t}


minds = minds.map(normalize_text)
minds['train'][0]['transcription'][:120]

Map:   0%|          | 0/19044 [00:00<?, ? examples/s]

Map:   0%|          | 0/4212 [00:00<?, ? examples/s]

Map:   0%|          | 0/4624 [00:00<?, ? examples/s]

'嗨'

## 4) Build CTC vocab (space→`|`)

In [ ]:
from collections import Counter


def collect_chars(ds, key='transcription'):
    c = Counter()
    for s in ds[key]:
        c.update(list(s))
    return c


cnt = Counter()
for sp in ['train', 'validation', 'test']:
    cnt.update(collect_chars(minds[sp]))
chars = sorted([ch for ch in cnt if ch != ' '])
vocab = {ch: i for i, ch in enumerate(chars)}
vocab['|'] = len(vocab)
vocab['<unk>'] = len(vocab)
vocab['<pad>'] = len(vocab)
VOCAB_DIR.mkdir(parents=True, exist_ok=True)
with open(VOCAB_DIR/'vocab.json', 'w', encoding='utf-8') as f:
    json.dump(vocab, f, ensure_ascii=False, indent=2)
len(vocab)

2579

## 5) Init tokenizer/processor & XLS-R-300M model

In [ ]:
CKPT = 'facebook/wav2vec2-xls-r-300m'
feature_extractor = AutoFeatureExtractor.from_pretrained(
    CKPT, return_attention_mask=True)
tokenizer = Wav2Vec2CTCTokenizer(str(
    VOCAB_DIR/'vocab.json'), unk_token='<unk>', pad_token='<pad>', word_delimiter_token='|')
processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)

model = AutoModelForCTC.from_pretrained(
    CKPT,
    ctc_loss_reduction='mean',
    mask_time_prob=0.05,    # 轻量时间掩码
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),

)
model.freeze_feature_extractor()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()
device

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1818: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


device(type='cuda')

## 6) Encode → input_values / attention_mask / labels

In [ ]:
def prepare_batch(batch):
    audio = batch['audio']
    ins = processor(
        audio['array'], sampling_rate=audio['sampling_rate'], return_attention_mask=True)
    batch['input_values'] = ins['input_values'][0]
    batch['attention_mask'] = ins['attention_mask'][0]
    with processor.as_target_processor():
        batch['labels'] = processor(batch['transcription']).input_ids
    return batch


encoded = minds.map(
    prepare_batch, remove_columns=minds['train'].column_names, num_proc=1)
encoded

Map:   0%|          | 0/19044 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Map:   0%|          | 0/4212 [00:00<?, ? examples/s]

Map:   0%|          | 0/4624 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_values', 'attention_mask', 'labels'],
        num_rows: 19044
    })
    validation: Dataset({
        features: ['input_values', 'attention_mask', 'labels'],
        num_rows: 4212
    })
    test: Dataset({
        features: ['input_values', 'attention_mask', 'labels'],
        num_rows: 4624
    })
})

## 7) Collator & Metrics (WER/CER) + Sanity check

In [ ]:
from dataclasses import dataclass
from typing import Union


@dataclass
# class DataCollatorCTCWithPadding:
#     processor: Wav2Vec2Processor
#     padding: Union[bool,str]='longest'
#     def __call__(self, features: List[Dict]):
#         inf = [{'input_values': f['input_values']} for f in features]
#         lab = [{'input_ids': f['labels']} for f in features]
#         batch = self.processor.pad(inf, padding=self.padding, return_tensors='pt')
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(lab, padding=self.padding, return_tensors='pt')
#         labels = labels_batch['input_ids'].masked_fill(labels_batch['attention_mask'].ne(1), -100)
#         batch['labels']=labels
#         return batch
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = "longest"   # True 等价于 'longest'，写字符串更直观

    def __call__(self, features: List[Dict]):
        # 1) 音频特征
        inputs = [{"input_values": f["input_values"]} for f in features]
        batch = self.processor.pad(
            inputs,
            padding=self.padding,
            return_tensors="pt",
            pad_to_multiple_of=8,
        )
        # 2) 文本 labels
        labels = [{"input_ids": f["labels"]} for f in features]
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                labels,
                padding=self.padding,
                return_tensors="pt",
                pad_to_multiple_of=8,
            )
        # 3) 把 label 的 padding 位改成 -100
        label_mask = labels_batch.get("attention_mask", None)
        label_ids = labels_batch["input_ids"]
        if label_mask is not None:
            label_ids = label_ids.masked_fill(label_mask.ne(1), -100)
        else:
            # 极少数情况下没有 attention_mask，就把 pad_token_id 直接置 -100
            pad_id = self.processor.tokenizer.pad_token_id
            label_ids = label_ids.masked_fill(label_ids.eq(pad_id), -100)

        batch["labels"] = label_ids
        return batch


data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

wer_metric = evaluate.load('wer')
cer_metric = evaluate.load('cer')


def _norm_str_list(xs):
    out = []
    for s in xs:
        s = s.strip()
        # 若是中文任务，通常不强制 lower；英文可视需要加：s = s.lower()
        s = " ".join(s.split())  # 合并多空格
        out.append(s)
    return out


def compute_metrics(pred):
    pred_ids = np.argmax(pred.predictions, axis=-1)

    # 把 -100 还原为 pad_token_id 才能正常 decode
    label_ids = pred.label_ids.copy()
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids,  skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    pred_str = _norm_str_list(pred_str)
    label_str = _norm_str_list(label_str)

    return {
        "wer": wer_metric.compute(predictions=pred_str, references=label_str),
        "cer": cer_metric.compute(predictions=pred_str, references=label_str),
    }
# def compute_metrics(pred):
#     pred_ids = np.argmax(pred.predictions, axis=-1)
#     label_ids = pred.label_ids.copy()
#     label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
#     pred_str  = processor.batch_decode(pred_ids,  skip_special_tokens=True)
#     label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
#     return {'wer': wer_metric.compute(predictions=pred_str, references=label_str),
#             'cer': cer_metric.compute(predictions=pred_str, references=label_str)}

# # sanity check
# sample = encoded['validation'].select(range(min(3, len(encoded['validation']))))
# if len(sample):
#     ins = processor.pad({'input_values': sample['input_values']}, padding=True, return_tensors='pt')
#     with torch.no_grad():
#         lg = model(input_values=ins['input_values'].to(device), attention_mask=ins['attention_mask'].to(device)).logits
#     hyp_ids = lg.argmax(dim=-1).cpu().numpy()
#     lbl_ids=[]
#     for seq in sample['labels']:
#         arr=np.array(seq, dtype=np.int64); arr[arr==-100]=processor.tokenizer.pad_token_id; lbl_ids.append(arr.tolist())
#     hyp = processor.batch_decode(hyp_ids, skip_special_tokens=True)
#     ref = processor.batch_decode(lbl_ids, skip_special_tokens=True)
#     for i,(r,h) in enumerate(zip(ref,hyp),1):
#         print(f'[{i}] REF: {r[:80]}')
#         print(f'[{i}] HYP: {h[:80]}')


# sanity check（确保 model 已 .eval() 且在正确 device）
model.eval()

sample = encoded["validation"].select(
    range(min(3, len(encoded["validation"]))))
if len(sample):
    # 正确的打包方式：列表[{"input_values": ...}, ...]
    feats = [{"input_values": iv} for iv in sample["input_values"]]
    ins = processor.pad(feats, padding=True, return_tensors="pt")

    ins = {k: v.to(device) for k, v in ins.items()}  # 送去同一 device
    with torch.no_grad():
        logits = model(input_values=ins["input_values"],
                       attention_mask=ins.get("attention_mask", None)).logits

    hyp_ids = logits.argmax(dim=-1).cpu().numpy()

    # 处理 labels：把 -100 还原为 pad_token_id 再 decode
    lbl_ids = []
    for seq in sample["labels"]:
        arr = np.array(seq, dtype=np.int64)
        arr[arr == -100] = processor.tokenizer.pad_token_id
        lbl_ids.append(arr.tolist())

    hyp = processor.batch_decode(hyp_ids, skip_special_tokens=True)
    ref = processor.batch_decode(lbl_ids, skip_special_tokens=True)

    for i, (r, h) in enumerate(zip(ref, hyp), 1):
        print(f"[{i}] REF: {r[:80]}")
        # print(f"[{i}] HYP: {h[:80]}")

[1] REF: 哎你好啊 你好啊你可以叫我玛丽
[2] REF: 好的
[3] REF: 那你大概 嗯你说


In [ ]:
vocab_size = processor.tokenizer.vocab_size
in_features = model.lm_head.in_features
old_out = model.lm_head.out_features

if old_out != vocab_size:
    print(f"[Fix] Rebuilding lm_head: {old_out} -> {vocab_size}")
    new_head = torch.nn.Linear(in_features, vocab_size, bias=True)
    # 可选：若 old_out > vocab_size，保留前 vocab_size 行的权重做“部分继承”
    with torch.no_grad():
        if old_out >= vocab_size:
            new_head.weight[:vocab_size].copy_(
                model.lm_head.weight[:vocab_size])
            new_head.bias[:vocab_size].copy_(model.lm_head.bias[:vocab_size])
        else:
            # old_out < vocab_size，拷贝已有部分，其余随机初始化即可
            new_head.weight[:old_out].copy_(model.lm_head.weight)
            new_head.bias[:old_out].copy_(model.lm_head.bias)
    model.lm_head = new_head.to(model.device)
    model.config.vocab_size = vocab_size

# 这些最好也设置好（一次即可）
if processor.tokenizer.pad_token is None and "<pad>" in processor.tokenizer.get_vocab():
    processor.tokenizer.pad_token = "<pad>"
if getattr(model.config, "pad_token_id", None) is None:
    model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.ctc_zero_infinity = True

# 再次强断言
head_out = model.lm_head.out_features
assert head_out == processor.tokenizer.vocab_size, f"mismatch: head={head_out}, vocab={processor.tokenizer.vocab_size}"
print(
    f"[OK] vocab_size={processor.tokenizer.vocab_size}, lm_head_out={head_out}")

[Fix] Rebuilding lm_head: 2581 -> 2579
[OK] vocab_size=2579, lm_head_out=2579


In [ ]:
if processor.tokenizer.pad_token is None and "<pad>" in processor.tokenizer.get_vocab():
    processor.tokenizer.pad_token = "<pad>"
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.ctc_zero_infinity = True
print("[OK] pad_token_id & ctc_zero_infinity set")

[OK] pad_token_id & ctc_zero_infinity set


In [ ]:

TARGET_SR = getattr(processor.feature_extractor, "sampling_rate", 16000)
x = minds["train"][0]["audio"]
assert x["sampling_rate"] == TARGET_SR and x["array"].ndim == 1
print("[OK] audio resampled to 16k mono")

[OK] audio resampled to 16k mono


In [ ]:
feats = [{"input_values": minds["validation"][i]["audio"]["array"]}
         for i in range(2)]
bat = processor.pad(feats, padding=True, return_tensors="pt")
assert bat["input_values"].ndim == 2 and "attention_mask" in bat
print("[OK] collator input structure & mask")

[OK] collator input structure & mask


In [ ]:
def clean_zh(s):
    return " ".join(s.strip().split())  # 你的规范化策略


with processor.as_target_processor():
    ids = processor([clean_zh(minds["train"][i]["transcription"])
                    for i in range(2)]).input_ids
assert isinstance(ids[0][0], int)
print("[OK] labels from as_target_processor & normalized")

[OK] labels from as_target_processor & normalized


In [ ]:
model.eval()
with torch.no_grad():
    lg = model(**{k: v.to(model.device)
               for k, v in bat.items() if k != "labels"}).logits
hyp_ids = lg.argmax(dim=-1).cpu().numpy()
_ = processor.batch_decode(hyp_ids, skip_special_tokens=True)
print("[OK] greedy decode runs")

[OK] greedy decode runs


In [ ]:
blank_id = getattr(model.config, "blank_token_id",
                   processor.tokenizer.vocab_size - 1)
br = (hyp_ids[0] == blank_id).mean()
print(f"[OK] blank_ratio≈{br:.2f}")

[OK] blank_ratio≈0.00


In [ ]:
import evaluate
import numpy as np
wer = evaluate.load("wer")
cer = evaluate.load("cer")
lbl = np.array([[processor.tokenizer.pad_token_id if x == -
               100 else x for x in y] for y in ids], dtype=object)
pred = processor.batch_decode(hyp_ids, skip_special_tokens=True)
ref = processor.batch_decode(lbl,     skip_special_tokens=True)
_wer, _cer = wer.compute(predictions=pred, references=ref), cer.compute(
    predictions=pred, references=ref)
print(f"[OK] metrics: WER={_wer:.3f}, CER={_cer:.3f}")

[OK] metrics: WER=1.000, CER=10.214


In [ ]:
tiny = encoded["train"].select(range(12))
args = TrainingArguments(output_dir="tmp_overfit", max_steps=300,
                         per_device_train_batch_size=3, learning_rate=3e-4,
                         logging_steps=20, save_steps=10_000, fp16=torch.cuda.is_available(), report_to='none',)
trainer = Trainer(model=model, args=args, train_dataset=tiny, data_collator=data_collator,
                  tokenizer=processor.feature_extractor)
trainer.train()
print("[OK] tiny overfit finished; loss should drop")

/tmp/ipython-input-1697839660.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args, train_dataset=tiny, data_collator=data_collator,
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss
20,151.275000
40,75.640500
60,34.952500
80,11.232500
100,5.476500
120,4.823000
140,4.713800
160,4.695300
180,4.657600
200,4.634900


[OK] tiny overfit finished; loss should drop


## 8) Train

In [ ]:
# #=======================================Round 1=======================================

# from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

# args = TrainingArguments(
#     output_dir=str(DATA_ROOT/'outputs'),
#     dataloader_num_workers=4,      # >0 才会有多进程加载
#     dataloader_pin_memory=True,    # GPU 传输更快
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=4,      # 有效 batch = 16
#     learning_rate=1e-4,                 # 若抖动大，改 5e-5
#     weight_decay=0.01,
#     warmup_ratio=0.1,
#     num_train_epochs=24.0,
#     lr_scheduler_type="cosine",         # 显式写出，虽然默认就是 linear
#     gradient_checkpointing=True,
#     fp16=torch.cuda.is_available(),
#     group_by_length=False,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     save_total_limit=2,
#     logging_steps=1000,
#     load_best_model_at_end=True,
#     metric_for_best_model='cer',
#     greater_is_better=False,
#     report_to='none',
#     optim="adamw_bnb_8bit",
# )

# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=encoded['train'],
#     eval_dataset=encoded['validation'],
#     # tokenizer=processor,
#     processing_class=processor,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(
#         early_stopping_patience=3,               # 连续3次评估无显著提升就停
#         early_stopping_threshold=1e-3            # CER 改善 < 0.001 视为不显著
#     )]
# )
# trainer.train()

In [25]:
# #=======================================Round 2=======================================

# best_ckpt = trainer.state.best_model_checkpoint
# print("best ckpt:", best_ckpt)

# from transformers import Wav2Vec2ForCTC
# model = Wav2Vec2ForCTC.from_pretrained(best_ckpt)   # 仅加载权重
# # 重新构建新的 TrainingArguments / Trainer（优化器会重新初始化）

# new_args = TrainingArguments(
#     output_dir=str(DATA_ROOT/'outputs_v2'),
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=4,        # 有效 batch 仍为 16
#     learning_rate=5e-5,                   # ← 从 1e-4 降一档（后半程更稳）
#     weight_decay=0.01,
#     # 用 epoch 续训 3~5 个；或改成 max_steps 也可
#     num_train_epochs=5.0,
#     lr_scheduler_type="cosine",           # 线性 warmup→线性衰减
#     warmup_ratio=0.1,                     # ← 用比例更方便（约 10% warmup）
#     gradient_checkpointing=True,
#     bf16=True,                            # A100 建议用 BF16（比 FP16 稳）
#     dataloader_num_workers=4,
#     dataloader_pin_memory=True,
#     group_by_length=False,                 # 续训阶段可开启，提升吞吐/省 pad
#     eval_strategy="steps",          # ← 官方参数名是 eval_strategy
#     eval_steps=1000,
#     save_steps=1000,
#     save_total_limit=2,                   # 只保留“最佳+最近”两个 ckpt
#     logging_steps=50,
#     load_best_model_at_end=True,
#     metric_for_best_model='cer',
#     greater_is_better=False,
#     report_to='none',
# )

# new_trainer = Trainer(
#     model=model,
#     args=new_args,
#     train_dataset=encoded["train"],
#     eval_dataset=encoded["validation"],
#     processing_class=processor,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# new_trainer.train()

In [ ]:
# !cp -r cs_dialogue/outputs_v2/checkpoint-5000/ '/content/drive/MyDrive/Project2-PRS/models'

In [ ]:
# #=======================================Round 3=======================================
# from transformers import Wav2Vec2ForCTC

# CKPT = "/content/drive/MyDrive/Project2-PRS/models/round2-checkpoint-5000"
# model = Wav2Vec2ForCTC.from_pretrained(CKPT)

# # === 3) 训练参数：cosine + warmup 10% + 早停；注意把 eval/save 策略写成官方键名 ===
# from transformers import EarlyStoppingCallback

# new_args_round3 = TrainingArguments(
#     output_dir=str(DATA_ROOT/'outputs_round3'),  # 新输出目录
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=4,               # 有效 batch=16
#     learning_rate=5e-5,                          # 二轮基础上再稳一点
#     weight_decay=0.01,
#     num_train_epochs=12.0,                        # 先续 3~5 个 epoch
#     lr_scheduler_type="cosine",                  # 余弦退火（Trainer 原生）
#     warmup_ratio=0.10,                           # 10% warmup
#     gradient_checkpointing=True,
#     bf16=True,                                   # A100 建议 bf16
#     dataloader_num_workers=4,
#     dataloader_pin_memory=True,

#     eval_strategy="epoch",
#     # eval_steps=1000,
#     save_strategy="epoch",
#     # save_steps=1000,
#     save_total_limit=2,

#     logging_steps=50,
#     load_best_model_at_end=True,
#     metric_for_best_model="cer",
#     greater_is_better=False,
#     report_to="none",

#     #8-bit optimizer
#     optim="adamw_bnb_8bit",
# )

# new_trainer = Trainer(
#     model=model,
#     args=new_args_round3,
#     train_dataset=encoded["train"],
#     eval_dataset=encoded["validation"],
#     processing_class=processor,                  # 你之前已改用 processing_class ✅
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(
#         early_stopping_patience=3,               # 连续3次评估无显著提升就停
#         early_stopping_threshold=1e-3            # CER 改善 < 0.001 视为不显著
#     )]
# )

In [28]:
# new_trainer.train()

In [ ]:
# #=======================================Round 3 (continue)=======================================
# # BEST = "/content/cs_dialogue/outputs_round3/checkpoint-14292"
# CKPT = "/content/drive/MyDrive/Project2-PRS/models/checkpoint-14292"


# from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer, EarlyStoppingCallback
# # 1) 仅加载模型权重（不会带上一次的优化器/调度器状态）
# model = Wav2Vec2ForCTC.from_pretrained(CKPT)

# # 2) 新一轮参数（示例：把 LR 小降；按 epoch 评估/保存，早停照旧）
# round3_coutinue = TrainingArguments(
#     output_dir=str(DATA_ROOT/'outputs_round3_cont'),
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=4,
#     learning_rate=1e-5,                 # ← 比 5e-5 再稳一点
#     weight_decay=0.01,
#     num_train_epochs=12.0,               # 再跑 1–2 个 epoch，交给早停兜底
#     lr_scheduler_type="cosine",
#     warmup_ratio=0,
#     gradient_checkpointing=True,
#     bf16=True,
#     dataloader_num_workers=4,
#     dataloader_pin_memory=True,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     save_total_limit=2,
#     logging_steps=1000,
#     load_best_model_at_end=True,
#     metric_for_best_model="cer",
#     greater_is_better=False,
#     report_to="none",
#     optim="adamw_bnb_8bit",             # 继续用 8-bit 优化器（已安装 bnb）
# )

# trainer = Trainer(
#     model=model,
#     args=round3_coutinue,
#     train_dataset=encoded["train"],
#     eval_dataset=encoded["validation"],
#     processing_class=processor,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=1e-3)]
# )

# trainer.train()  # ← 不要传 resume_from_checkpoint

In [ ]:

from pyctcdecode import build_ctcdecoder

# 1) 严格按 id 构造 labels，避免 get_vocab() 的无序/多余项
vocab_size = processor.tokenizer.vocab_size
labels = [processor.tokenizer.convert_ids_to_tokens(
    i) for i in range(vocab_size)]

# 2) CTC 规则：pad 作为 blank；word_delimiter 作为空格
pad_id = processor.tokenizer.pad_token_id
if pad_id is not None:
    labels[pad_id] = ""   # CTC blank

# 有的 tokenizer 定义了 word_delimiter_token / 其 id；常见是 "|"
if hasattr(processor.tokenizer, "word_delimiter_token_id") and \
   processor.tokenizer.word_delimiter_token_id is not None:
    labels[processor.tokenizer.word_delimiter_token_id] = " "

# 3) 保险起见再断言一次，必须与 logits 维度一致
assert len(labels) == model.lm_head.out_features, \
    f"labels={len(labels)} vs lm_head={model.lm_head.out_features}"

# 4) 构建 decoder（无 LM；有 KenLM 时加 kenlm_model_path=...）
decoder = build_ctcdecoder(labels)

print("vocab_size =", processor.tokenizer.vocab_size)
print("lm_head_out =", model.lm_head.out_features)
assert processor.tokenizer.vocab_size == model.lm_head.out_features, "vocab != lm_head"

wer_metric = evaluate.load('wer')
cer_metric = evaluate.load('cer')


def _norm(xs):
    out = []
    for s in xs:
        s = " ".join(s.strip().split())
        out.append(s)
    return out


def compute_metrics_beam(pred):
    # pred.predictions: [B, T, V] 的 logits (numpy)
    logits = pred.predictions
    # 逐条 beam 解码（示例 beam_width=50）
    pred_str = [decoder.decode(logit, beam_width=50) for logit in logits]

    # 还原 -100 并 decode 参考
    label_ids = pred.label_ids.copy()
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    pred_str = _norm(pred_str)
    label_str = _norm(label_str)
    return {
        "wer": wer_metric.compute(predictions=pred_str, references=label_str),
        "cer": cer_metric.compute(predictions=pred_str, references=label_str),
    }


# 取一个很小的 batch（比如 test 的前 2~4 条）
small = encoded["test"].select(range(min(4, len(encoded["test"]))))

batch = data_collator([{k: v for k, v in small[i].items()}
                      for i in range(len(small))])
with torch.no_grad():
    lg = model(input_values=batch["input_values"].to(model.device),
               attention_mask=batch["attention_mask"].to(model.device)).logits
lg = lg.cpu().numpy()  # [B, T, V]

# ① labels 长度 = logits 末维 = lm_head_out
print("logits.shape =", lg.shape)
assert lg.shape[-1] == len(
    labels) == model.lm_head.out_features, "decoder labels dim mismatch"

# ② 解码一条，确保不包含分隔符“|”或 pad 符号名
out = decoder.decode(lg[0], beam_width=50)
print("beam sample:", out[:120])
assert "|" not in out, "decoded text still contains '|' (word delimiter not mapped to space)"
if processor.tokenizer.pad_token is not None:
    assert processor.tokenizer.pad_token not in out, "decoded text contains PAD token"
print("[OK] labels/logits 对齐 & 空格/blank 映射正常")

vocab_size = 2579
lm_head_out = 2579
logits.shape = (4, 330, 2579)
beam sample: HELLO HELLO 很好兴认识你呀
[OK] labels/logits 对齐 & 空格/blank 映射正常


In [ ]:
# BEST = "/content/cs_dialogue/outputs_round3/checkpoint-14292"  # 你JSON里给的
# from transformers import Wav2Vec2ForCTC, Trainer, TrainingArguments

# model = Wav2Vec2ForCTC.from_pretrained(BEST)
# args = TrainingArguments(output_dir=str(DATA_ROOT/'final_eval'), report_to='none')
# tester = Trainer(
#     model=model, args=args,
#     eval_dataset=encoded["test"],        # ← 用 test split
#     processing_class=processor,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,     # 仍计算 CER/WER（主：CER）
# )
# test_metrics = tester.evaluate(metric_key_prefix="test")
# print(test_metrics)  # 建议保存到文件
# # 保存“可部署包”
# tester.save_model(str(DATA_ROOT/'final_best'))
# processor.save_pretrained(str(DATA_ROOT/'final_best'))

In [ ]:
# BEST = "/content/cs_dialogue/outputs_round3_cont/checkpoint-8337"  # 你JSON里给的
# from transformers import Wav2Vec2ForCTC, Trainer, TrainingArguments

# model = Wav2Vec2ForCTC.from_pretrained(BEST)
# args = TrainingArguments(output_dir=str(DATA_ROOT/'final_eval_8337'), report_to='none')
# tester = Trainer(
#     model=model, args=args,
#     eval_dataset=encoded["test"],        # ← 用 test split
#     processing_class=processor,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics_beam,     # 仍计算 CER/WER（主：CER）
# )
# test_metrics = tester.evaluate(metric_key_prefix="test")
# print(test_metrics)  # 建议保存到文件
# # 保存“可部署包”
# tester.save_model(str(DATA_ROOT/'final_best_8337'))
# processor.save_pretrained(str(DATA_ROOT/'final_best_8337'))

In [ ]:
from transformers import Wav2Vec2ForCTC, Trainer, TrainingArguments
from google.colab import drive
drive.mount('/content/drive')

BEST = "/content/drive/MyDrive/Project2-PRS/models/checkpoint-8337"  # 你JSON里给的

model = Wav2Vec2ForCTC.from_pretrained(BEST)
args = TrainingArguments(output_dir=str(
    DATA_ROOT/'final_eval_8337_beam'), report_to='none')
tester = Trainer(
    model=model, args=args,
    eval_dataset=encoded["test"],        # ← 用 test split
    processing_class=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics_beam,     # 仍计算 CER/WER（主：CER）
)
test_metrics = tester.evaluate(metric_key_prefix="test_beam")
print(test_metrics)  # 建议保存到文件
# 保存“可部署包”
tester.save_model(str(DATA_ROOT/'final_best_8337_beam'))
processor.save_pretrained(str(DATA_ROOT/'final_best_8337_beam'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'test_beam_loss': 1.1411978006362915, 'test_beam_model_preparation_time': 0.0061, 'test_beam_wer': 0.6102945698296538, 'test_beam_cer': 0.2732353123131174, 'test_beam_runtime': 829.6704, 'test_beam_samples_per_second': 5.573, 'test_beam_steps_per_second': 0.697}


[]

In [ ]:
# =======================================Round 4(LoRA)=======================================
import math
import os
from transformers import TrainerCallback
from transformers import BitsAndBytesConfig
from peft import LoraConfig
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, EarlyStoppingCallback
from transformers import Wav2Vec2ForCTC


# 从 Round 2 的“最佳 checkpoint”继续（仍然只加载权重）
CKPT = "/content/drive/MyDrive/Project2-PRS/models/checkpoint-8337"


model = Wav2Vec2ForCTC.from_pretrained(
    CKPT,
    # quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    # device_map="auto",
)

# === 1) 配置 LoRA（对自注意力投影层加 LoRA）===
# *注意*：Wav2Vec2 编码器的注意力层常见投影名为 q_proj/k_proj/v_proj/out_proj
lora_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"]
)

ADAPTER_NAME = "lora_wav2vec2_round4"
ADAPTER_BEST_DIR = str(
    (DATA_ROOT / "outputs_round4_lora_adapter_best").resolve())
model.add_adapter(lora_config, adapter_name=ADAPTER_NAME)  # ← 官方推荐调用
model.set_adapter(ADAPTER_NAME)                          # 训练时启用该 adapter

# model.print_trainable_parameters()  # 调试：应显示仅少量参数可训练


class BestAdapterSaver(TrainerCallback):
    def __init__(self, adapter_name: str, out_dir: str, metric_name="eval_cer", greater_is_better=False):
        self.adapter_name = adapter_name
        self.out_dir = out_dir
        self.metric_name = metric_name
        self.sign = 1.0 if greater_is_better else -1.0
        self.best_score = -math.inf  # 用带符号的分数做“越大越好”的统一比较
        os.makedirs(out_dir, exist_ok=True)

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is None or self.metric_name not in metrics:
            return
        score = self.sign * metrics[self.metric_name]
        if score > self.best_score:
            self.best_score = score
            # 只保存 LoRA 适配器（不保存全模型）
            adapter_dir = os.path.join(
                self.out_dir, f"best_adapter_step-{state.global_step}")
            # kwargs["model"].save_adapter(adapter_dir, adapter_name=self.adapter_name)
            kwargs["model"].save_pretrained(adapter_dir)
            # 也可额外写一个“latest-best”软链接/拷贝，方便复用：
            latest_dir = os.path.join(self.out_dir, "best_adapter_latest")
            try:
                if os.path.islink(latest_dir) or os.path.exists(latest_dir):
                    import shutil
                    shutil.rmtree(latest_dir)
                import shutil
                shutil.copytree(adapter_dir, latest_dir)
            except Exception:
                pass
        return


lora_args = TrainingArguments(
    output_dir=str(DATA_ROOT/'outputs_round4_lora'),
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,               # LoRA 常用更大的起始 LR（5e-4 ~ 1e-3）
    weight_decay=0.01,
    num_train_epochs=7.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.10,
    gradient_checkpointing=False,
    bf16=True,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,

    # ★ 改为按 epoch
    eval_strategy="epoch",
    save_strategy="no",
    save_total_limit=2,

    logging_steps=200,
    load_best_model_at_end=False,
    metric_for_best_model="cer",
    greater_is_better=False,
    report_to="none",
    remove_unused_columns=False,
)


lora_trainer = Trainer(
    model=model,                                # LoRA 包裹后的 PeftModel
    args=lora_args,
    train_dataset=encoded["train"],
    eval_dataset=encoded["validation"],
    processing_class=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=1e-3),
               BestAdapterSaver(adapter_name=ADAPTER_NAME,
                                out_dir=ADAPTER_BEST_DIR,
                                metric_name="eval_cer",
                                greater_is_better=False),
               ]
)

# 与 Round 2 一样：这是“从权重开始新一轮”，不要恢复旧 optimizer/scheduler
lora_trainer.train()

ADAPTER_OUT = str(DATA_ROOT/'outputs_round4_lora_adapter')
# model.save_adapter(ADAPTER_OUT, adapter_name=ADAPTER_NAME)
model.save_pretrained(ADAPTER_OUT)  # ← 同上，用 save_pretrained

# 之后可在任意同架构模型上：
# model.load_adapter(ADAPTER_OUT, adapter_name="lora_round3")
# model.set_adapter("lora_round3")

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_proces

Epoch,Training Loss,Validation Loss,Wer,Cer
1,0.514000,1.072975,0.566141,0.230271
2,0.520000,1.064852,0.566534,0.230248
3,0.504000,1.063225,0.568822,0.231389
4,0.491200,1.040827,0.565945,0.228478
5,0.497400,1.041942,0.566207,0.228478
6,0.504400,1.053439,0.565553,0.227976
7,0.525000,1.051696,0.565226,0.228079


/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
BASE = "/content/drive/MyDrive/Project2-PRS/models/checkpoint-8337"
BEST_ADAPTER_DIR = str(
    (DATA_ROOT / "outputs_round4_lora_adapter_best" / "best_adapter_latest").resolve())

model = Wav2Vec2ForCTC.from_pretrained(BASE)
model.load_adapter(BEST_ADAPTER_DIR, adapter_name="lora_best")
model.set_adapter("lora_best")
# 然后用 Trainer 做 evaluate / 继续训练即可

In [ ]:
# from transformers import Wav2Vec2ForCTC
# from google.colab import drive
# drive.mount('/content/drive')

# #载入你当前最佳权重（全量权重）
# CKPT = "/content/drive/MyDrive/Project2-PRS/models/checkpoint-8337"
# model = Wav2Vec2ForCTC.from_pretrained(CKPT)

# # 给自注意力投影层加 LoRA（按你的 Wav2Vec2 命名可能是 q_proj/k_proj/v_proj/out_proj）
# lora_cfg = LoraConfig(
#     r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
#     target_modules=["q_proj","k_proj","v_proj","out_proj"]
# )
# ADAPTER_NAME = "lora_round4"
# model.add_adapter(lora_cfg, adapter_name=ADAPTER_NAME)   # 官方推荐 API
# model.set_adapter(ADAPTER_NAME)

# lora_args = TrainingArguments(
#     output_dir=str(DATA_ROOT/'outputs_round4_lora'),
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=4,
#     learning_rate=5e-4,                # LoRA 可用更大 LR
#     weight_decay=0.01,
#     num_train_epochs=7.0,
#     lr_scheduler_type="cosine",
#     warmup_ratio=0.10,
#     bf16=True,
#     gradient_checkpointing=True,
#     dataloader_num_workers=4,
#     dataloader_pin_memory=True,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="cer",
#     greater_is_better=False,
#     logging_steps=50,
#     report_to="none",
# )

# lora_trainer = Trainer(
#     model=model,
#     args=lora_args,
#     train_dataset=encoded["train"], eval_dataset=encoded["validation"],
#     processing_class=processor, data_collator=data_collator,
#     compute_metrics=compute_metrics,   # 建议直接用上面“beam版”评估
# )
# lora_trainer.train()

# # 仅保存适配器，便于复用/分享
# model.save_adapter(str(DATA_ROOT/'outputs_round4_lora_adapter'), adapter_name=ADAPTER_NAME)

In [ ]:
# # 1) 最终在 test 上做指标评估（会返回 metrics）
# test_metrics = trainer.evaluate(eval_dataset=encoded["test"])
# print("TEST (evaluate):", test_metrics)